In [6]:
import cv2
import math
import data_process_ml

import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.hub import load_state_dict_from_url
import sklearn
import pandas as pd
import copy

from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split
from torchvision.datasets import VisionDataset
import os
from torch import optim

In [1]:
import data_process_ml
morpho_path = ".\AneuX\data-v1.0\data\morpho-per-cut.csv"
patient_path = ".\AneuX\data-v1.0\data\clinical.csv"
morpho_data_patient = data_process_ml.read_and_combine_data(morpho_path,patient_path)
merged_dataset = data_process_ml.encode_column(morpho_data_patient)
merged_dataset = data_process_ml.drop_columns(merged_dataset)
morpho_data_cut1,morpho_data_dome = data_process_ml.output_cut1anddome(merged_dataset)

d:\Universityofleeds\MSC_PROG\Msc_project\code\data_process_ml.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(morpho_data_cut1.columns[3:23], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\data_process_ml.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(['source_x',"cuttype","dataset"], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\data_process_ml.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [4]:
morpho_data_cut1

,writhe-original-H,writhe.1-original-mean,writhe.2-original-std,writhe.3-L1norm-H,writhe.4-L1norm-mean,writhe.5-L1norm-std,writhe.6-L2-H,writhe.7-L2-mean,writhe.8-L2-std,zmi_surf-0.0-0.0,...,zmi_surf.135-energy-n15,zmi_surf.136-energy-n16,zmi_surf.137-energy-n17,zmi_surf.138-energy-n18,zmi_surf.139-energy-n19,zmi_surf.140-energy-n20,age,sex_female,sex_male,status_ruptured
0,3.13602,0.686535,6.38481,-1.60037,0.330408,0.0534176,1.23964,6.05006,0.881166,0.891258,...,43.646,45.5701,47.4192,49.5589,52.4916,54.9455,64.2,1.0,0.0,0.0
4,2.28995,-0.123443,2.53599,-1.17942,0.312721,0.0778531,1.33947,4.88757,1.00908,1.071280,...,38.9036,42.1663,44.2741,46.3791,47.4028,49.1238,72.7,1.0,0.0,0.0
8,2.32917,-0.00466042,2.70505,-1.51013,0.212483,0.0667921,1.6863,5.88084,1.56453,1.435430,...,35.6309,37.6116,39.7437,41.7172,44.1529,45.8037,50.9,0.0,1.0,0.0
12,2.52705,-0.021154,3.17756,-1.49203,0.251283,0.0746368,1.82116,7.80336,1.92642,1.272820,...,38.4547,40.9813,43.2989,45.1772,47.5794,49.8087,47.3,1.0,0.0,1.0
16,2.98359,-0.395319,5.1354,-1.43807,0.349662,0.061156,1.167,5.40205,0.851545,0.822235,...,51.9063,55.152,58.0832,59.6127,60.9068,62.4867,47.3,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,1.87399,-0.00474863,1.65733,-1.17324,0.253628,0.0834412,1.42177,3.83603,1.08127,1.269270,...,33.1295,35.0355,37.64,40.1607,42.3614,43.8509,43.0,1.0,0.0,0.0
2760,1.5737,0.00373571,1.3765,-1.41009,0.218476,0.0835751,1.69978,5.23269,1.65098,1.378590,...,34.7983,37.8419,40.0753,41.8086,43.2362,45.5748,67.0,0.0,1.0,1.0
2764,2.46928,0.277979,3.02867,-1.34477,0.351678,0.0659528,1.2976,5.8804,0.980592,1.060260,...,39.9266,41.9626,44.2732,47.1096,50.0711,52.9587,84.0,1.0,0.0,1.0
2768,1.96354,0.0263473,1.87208,-1.51831,0.18834,0.0830219,1.7068,5.97952,2.06504,1.340760,...,35.7781,39.4211,40.5627,42.223,43.9946,46.1836,59.0,1.0,0.0,1.0


In [14]:
# read data

class AneuxDataset_Morph(Dataset):

    def __init__(self, df, transform = None):

        self.transform = transform
        self.df = df
        self.label = df["status_ruptured"].copy()
        self.label = torch.from_numpy(np.array(self.label)) 
        self.raw_data = copy.deepcopy(df)
        self.raw_data.drop(("status_ruptured"),axis=1,inplace=True)
        self.my_device = "cuda:0"
    
    def __getitem__(self, index):
        
        """ Returns one data pair (image and target caption). """
        data = self.df[index]
        label = self.label[index]
        #print(type(image))
        if self.transform is not None:      
            data= self.transform(data)
        return data,label

    def __len__(self):
                                
        return len(self.label)

load data

In [19]:
Morph_data = AneuxDataset_Morph(morpho_data_cut1)
train_size = int(len(Morph_data) * 0.8) # 80% training data
valid_size = len(Morph_data) - train_size
train_data, valid_data = random_split(Morph_data, [train_size, valid_size])

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=16,
    shuffle=True,
    #num_workers=2, 
    pin_memory=True
)

valid_loader = torch.utils.data.DataLoader(
    valid_data,
    batch_size=100, # Forward pass only so batch size can be larger
    shuffle=False,
    #num_workers=2, 
    pin_memory=True
)

In [23]:
len(train_data)

588

# A simple MLP MODEL with 2 FCL

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPModel, self).__init__()

        # Define layers
        self.Flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        
        self.loss_fn = nn.CrossEntropyLoss()
        #Adam
        #self.optimizer = optim.Adam(self.parameters(), lr=0.001) #weight decay
        self.optimizer = optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
        self.running_loss = 0
        self.loss = None
        self.losses = []

    def forward(self, x, target =None):
        # Forward pass
        x = self.Flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x
    def fit(self, x, targets):
        #train/optimize/fit
        preds = self.forward(x)
        self.loss = self.loss_fn(preds, targets)
        self.loss.backward()
        self.optimizer.step()
        
        loss_item = self.loss.item()
        return loss_item


    def reset_loss(self, value):
        self.running_loss = value
        self.losses = []
        return

# CNN MODEL 3 CONV & 3 FCL

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, input_size,input_channel, kernel_size, middle_channel, output_channel,hidden_input,output_size,padding = 0, Drop = 0.1):
        super(CNNModel, self).__init__()
        reduce_size = 0
        size = input_size
        if padding == 0:
            reduce_size = size - kernel_size +1
            reduce_size = reduce_size/2
            reduce_size = reduce_size - kernel_size +1
            reduce_size = int(reduce_size/2)
        else:
            reduce_size = int(size/4)
        # Define layers
        self.Drop = Drop
        self.Dropout = nn.Dropout(p=self.Drop)
        self.Conv2d1 = nn.Conv2d(in_channels=input_channel,out_channels=middle_channel, kernel_size=kernel_size,padding = padding)
        self.ReLU = nn.ReLU()
        self.MaxPool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Conv2d2 = nn.Conv2d(in_channels=middle_channel,out_channels=output_channel, kernel_size=kernel_size,padding = padding)
        self.Flatten = nn.Flatten()
        self.fc1 = nn.Linear(output_channel*reduce_size*reduce_size,hidden_input)
        self.fc2 = nn.Linear(hidden_input,output_size)
        
        self.loss_fn = nn.CrossEntropyLoss()
        #self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        self.optimizer = optim.SGD(self.parameters(), lr=0.001,momentum=0.9) 
        self.running_loss = 0
        self.loss = None
        self.losses = []
        self.dropout_enable = False
        
    def forward(self, x, target = None):

            
        x = self.Conv2d1(x)
        x = self.ReLU(x)
        
        if self.dropout_enable:
            x = self.Dropout(x)
            
        x = self.MaxPool2d(x)
        x = self.Conv2d2(x)
        x = self.ReLU(x)
        x = self.MaxPool2d(x)
        x = self.Flatten(x)
        x = self.fc1(x)
        x = self.ReLU(x)
        if self.dropout_enable:
            x = self.Dropout(x)
        x = self.fc2(x)

        #update
        
        return x


    def fit(self, x, targets):
        #train/optimize/fit
        preds = self.forward(x)
        self.loss = self.loss_fn(preds, targets)
        self.loss.backward()
        self.optimizer.step()
        
        loss_item = self.loss.item()
        return loss_item


    def reset_loss(self):
        self.running_loss = 0
        self.loss = None
        return

# RNN Model

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size,input_channel, kernel_size, middle_channel, output_channel,hidden_input,output_size,padding = 0, Drop = 0.1):
        super(CNNModel, self).__init__()
        self.LSTM = nn.LSTM()
        self.RNN = nn.GLU()

        
    def forward(self, x, target = None):

            
        x = self.Conv2d1(x)
        x = self.ReLU(x)
        
        if self.dropout_enable:
            x = self.Dropout(x)
            
        x = self.MaxPool2d(x)
        x = self.Conv2d2(x)
        x = self.ReLU(x)
        x = self.MaxPool2d(x)
        x = self.Flatten(x)
        x = self.fc1(x)
        x = self.ReLU(x)
        if self.dropout_enable:
            x = self.Dropout(x)
        x = self.fc2(x)

        #update
        
        return x


    def fit(self, x, targets):
        #train/optimize/fit
        preds = self.forward(x)
        self.loss = self.loss_fn(preds, targets)
        self.loss.backward()
        self.optimizer.step()
        
        loss_item = self.loss.item()
        return loss_item


    def reset_loss(self):
        self.running_loss = 0
        self.loss = None
        return